# 이름/학번

이름: 홍진우

학번: 20205276

In [2]:
import numpy as np
import torch

# Example dataset

강의를 위해서 임의의 dataset을 준비하겠습니다.
예제로 봐주시고, 큰 물리적 의미는 부여하지 않겠습니다.

- Data는 장미과와 국화과의 A 효소, B 효소, C 효소, D 효소를 측정한 값이라고 가정합니다.
- Label은 각 sample이 장미인지 (0) 국화인지 (1)에 대한 정보라고 가정합니다.

In [6]:
batch_size = 10
num_feature = 4
torch.manual_seed(0)

X_batch = torch.randn(batch_size, num_feature)
Y_batch = (torch.sum(X_batch, dim=1)>0).type(torch.float).reshape(-1,1)

In [7]:
X_batch

tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530],
        [-0.2159, -0.7425,  0.5627,  0.2596],
        [-0.1740, -0.6787,  0.9383,  0.4889],
        [ 1.2032,  0.0845, -1.2001, -0.0048],
        [-0.5181, -0.3067, -1.5810,  1.7066]])

In [9]:
#0이면 장미, 1이면 국화
Y_batch

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.]])

# Notation 정리

강의자료와 비교하면 

\begin{align*}
X_\text{batch} = 
\begin{bmatrix}
(x^{(1)})^\top\\
(x^{(2)})^\top\\
\vdots \\
(x^{(m)})^\top
\end{bmatrix}, \quad
Y_\text{batch} = 
\begin{bmatrix}
y^{(1)}\\
y^{(2)}\\
\vdots \\
y^{(m)}
\end{bmatrix}
\end{align*}

# Example Problem: Single Neuron

- 한개의 neuron이 있다고 가정하고 $\mathbb{R}^4$ 를 입력받아서 $\mathbb{R}$로 출력한다고 가정합니다.
- Activation 함수는 ReLU 함수, 즉, 
\begin{align*}
\text{ReLU}(x) = \max(0, x)
\end{align*}
를 사용한다고 가정합니다.

Neuron을 통해서 batch sample 전체를
\begin{align*}
Z = \begin{bmatrix}
(w^T x^{(1)} + b)^T \\
(w^T x^{(2)} + b)^T \\
\vdots \\
(w^T x^{(m)} + b)^T
\end{bmatrix} 
\end{align*}
연산을 수행해서 $Z$를 구하세요.

- $w$는 random Gaussian으로 생성하세요. 위에서 예기한 입력과 출력이 맞도록 weight를 생성하세요.
- Bias $b$는 1로 설정




In [12]:
# 가우시안 랜덤분포로 가중치 설정
W = torch.randn(num_feature, 1)
#bias
b = 1
# z = wx + b
Z = torch.matmul(X_batch, W)+b
print(Z)

tensor([[ 3.0662],
        [ 1.6351],
        [ 1.1136],
        [ 0.7891],
        [ 2.6685],
        [-0.6325],
        [ 1.5096],
        [ 1.3267],
        [-0.6050],
        [-0.3120]])


원하는 연산을 하는지 확인하도록 합니다

In [18]:
z_loop = torch.empty(batch_size, 1)

for i in torch.arange(batch_size):
    z_loop[i,:] = torch.matmul(W.T, X_batch[i,:].T)+b
    
print(z_loop)

tensor([[ 3.0662],
        [ 1.6351],
        [ 1.1136],
        [ 0.7891],
        [ 2.6685],
        [-0.6325],
        [ 1.5096],
        [ 1.3267],
        [-0.6050],
        [-0.3120]])


## Problem 1
ReLU 함수를 작성하고 위에서 찾은 $Z$의 각 원소에 ReLU 함수를 적용하여 `a`라는 변수에 저장하세요.

- torch.clamp() 함수를 공부하고 적용하세요




In [26]:
# 답 작성:
def ReLU(x):
    #np.maximum사용, max대비 행렬연산에 더 좋음
    #return np.maximum(0,x)
    
    #torch.clamp()는 범위 지정.
    return torch.clamp(x,min=0)

A = ReLU(Z)

print(A)

tensor([[3.0662],
        [1.6351],
        [1.1136],
        [0.7891],
        [2.6685],
        [0.0000],
        [1.5096],
        [1.3267],
        [0.0000],
        [0.0000]])


# Single Layer Network

- 한개의 Layer에 $k=5$개의 Neuron 이 있는 network를 구성하고 출력을 구하세요
- Activation function은 모든 neuron에 ReLU를 적용합니다
- 모든 weight는 Gaussian 분포로 랜덤 생성하세요 `torch.randn()`
- $i$ 번째 neuron의 weight들을 $w_i$라고 할때,
\begin{align*}
\text{W} = \begin{bmatrix}
w_1, w_2, w_3, w_4, w_5
\end{bmatrix}
\end{align*}
라고 하고, weight matrix `W`를 만드세요.
  - `W = torch.randn(???, ???)` 으로 생성
- Bias 역시 `b`라는 `tensor`에 저장하고, 각 neuron 별로 `1`로 설정합니다
  - `b = torch.ones(???,???)`
- 아래 problem 2-2에서 수업에서 배운 $Z$ 행렬과 $A$ 행렬을 구하세요

# Problem 2-1
`Z` 행렬과 `A` 행렬의 차원은 어떻게 나와야하나요?

답 작성)
Z와 A의 차이는 활성화 함수를 통해 음수를 없엔 것이므로 차원의 차이는 없다.

Z 는 (10,5), 


A 는 (10,5)

# Problem 2-2 
위에서 요구한 코딩을 완성하세요

In [41]:
# 답 작성
#10 by 4의 X_batch그대로 받아옴

#W는 랜덤하게 생성
W = torch.randn(num_feature,5)

#bias를 k의 개수에 맞게,즉 뉴런별로 생성
b = torch.ones(5)

#곱해주고 bias더함
Z = torch.matmul(X_batch, W)+b

#Activation Function
A = ReLU(Z)

print(A)

tensor([[2.9194, 0.6712, 0.3797, 3.0094, 0.8064],
        [1.4650, 4.0738, 0.0000, 1.9831, 2.7174],
        [1.5559, 0.7344, 2.4332, 2.6213, 0.5947],
        [0.0000, 1.2050, 2.0044, 0.0000, 2.7569],
        [1.8688, 0.0000, 2.1738, 2.3409, 0.3636],
        [2.0451, 0.0000, 2.3095, 2.3790, 0.0000],
        [0.9045, 0.3954, 2.1593, 1.5614, 1.0787],
        [0.2249, 0.0947, 2.7885, 1.0601, 1.2692],
        [2.2619, 2.1101, 0.0000, 1.9378, 0.0000],
        [2.8238, 0.0000, 0.0000, 0.9938, 0.0000]])


# Problem 3: 

- $x^{(3)}$ 를 입력으로하는 2번째 Neuron의 결과값을 출력하세요
- 위에서 구한 `A[i, j]`  인덱싱을 통해서 출력하세요
- `Python`의 인덱싱은 `0`부터 시작한다는 것을 주의하세요

In [60]:
# 답 작성
print("X 는 \n", X_batch)
print("이고, X3은 : \n", X_batch[2])

print("\n\nW 는 :\n", W)
print("이때 2번째 뉴런이므로, \n", W[:,1])

print("\n\nX의 3열, 그리고 W의 2행이 곱해진 부분은 : \n", A[2,1])

print("\n\nAnswer : ", A[2,1])

X 는 
 tensor([[-1.1258, -1.1524, -0.2506, -0.4339],
        [ 0.8487,  0.6920, -0.3160, -2.1152],
        [ 0.3223, -1.2633,  0.3500,  0.3081],
        [ 0.1198,  1.2377,  1.1168, -0.2473],
        [-1.3527, -1.6959,  0.5667,  0.7935],
        [ 0.5988, -1.5551, -0.3414,  1.8530],
        [-0.2159, -0.7425,  0.5627,  0.2596],
        [-0.1740, -0.6787,  0.9383,  0.4889],
        [ 1.2032,  0.0845, -1.2001, -0.0048],
        [-0.5181, -0.3067, -1.5810,  1.7066]])
이고, X3은 : 
 tensor([ 0.3223, -1.2633,  0.3500,  0.3081])


W 는 :
 tensor([[-0.2340,  0.7073,  0.5800,  0.2683, -0.0807],
        [-0.9801,  0.0605, -0.4890, -1.5771,  0.3609],
        [-1.3533, -0.2071,  1.4628, -0.6204,  0.9884],
        [-0.4322, -1.1187,  0.3784, -0.7804, -0.8739]])
이때 2번째 뉴런이므로, 
 tensor([ 0.7073,  0.0605, -0.2071, -1.1187])


X의 3열, 그리고 W의 2행이 곱해진 부분은 : 
 tensor(0.7344)


Answer :  tensor(0.7344)


# Problem 4: Multi-Layer network

- 3개의 layer가 있는 network를 구성합니다.
- 2번째 layer의 입력 크기는 $k^{[1]}=16$, 출력 크기는 $k^{[2]}=6$
- 마지막 layer의 출력은 $k^{[3]}=1$개의 neuron으로 구성
- 각 layer의 연산값을 구하세요. 
  - 각 layer의 선형 변환 결과값은 `Z1`, `Z2`, `Z3`로 저장하세요
  - 각 layer의 결과값은 `A1`, `A2`, `A3`로 저장하세요
- 모든 weight는 Gaussian 랜덤 변수로 생성, bias는 1로 구성된 벡터로 생성함
- 각 layer의 weight는 `W1`, `W2`, `W3`로하고, bias는 `b1`, `b2`, `b3`로 생성함
- Activation 함수는 `ReLU`를 적용하세요
- Loop 없이 행렬연산으로 구생하세요

In [87]:
# 답 작성

#가우시안 분포를 따르는 가중치 3개 선언
#k=16인 퍼셉트론 10by4 X 4by16
W1= torch.randn(num_feature,16)
#W1,Z1을 거쳐서 나온 10 by 16 에 대해, k=6인 퍼셉트론 => 10by16 X 16by6
W2= torch.randn(16,6)
#W2, Z2을 거쳐서 나온 10 by 6 에 대해, k=1인 퍼셉트론 => 10by6 X 6by1 => 10 by 1
W3= torch.randn(6,1)

#bias3개
b1= torch.ones(16)
b2= torch.ones(6)
b3= torch.ones(1)


Z1 = torch.matmul(X_batch, W1)+b1
Z2 = torch.matmul(Z1,W2)+b2
Z3 = torch.matmul(Z2,W3)+b3

A1 = ReLU(Z1)
A2 = ReLU(Z2)
A3 = ReLU(Z3)

print(A3)

tensor([[ 0.0000],
        [ 1.8711],
        [17.7120],
        [ 0.0000],
        [ 0.0000],
        [41.9245],
        [ 0.0000],
        [ 0.0000],
        [36.3505],
        [ 8.5659]])


In [88]:
print(Z1.shape)
print(Z1)


torch.Size([10, 16])
tensor([[ 3.2936e+00,  1.4234e+00,  9.9502e-01,  1.8896e+00, -5.9266e-01,
          2.2821e+00,  1.4739e+00,  1.7884e+00,  8.4252e-01, -9.0953e-01,
         -2.1053e+00, -1.6080e+00,  9.1458e-01,  7.3527e-01,  2.3951e+00,
         -7.8773e-03],
        [ 8.8832e-01,  2.3633e+00, -1.5688e+00,  3.8323e+00,  2.7277e+00,
          5.7278e-01, -3.5222e+00,  2.3670e+00,  1.1328e+00,  7.0061e+00,
          1.3683e+00,  5.6966e-01,  1.2853e+00,  4.0042e+00,  6.3783e-01,
          2.0976e-01],
        [ 3.4141e+00,  1.7901e+00,  6.8572e-01,  2.0432e-01,  2.4525e-01,
         -5.9159e-01,  1.4705e+00,  8.9042e-01, -3.7370e-01,  1.3852e-01,
          1.8280e-01,  2.9946e+00,  2.0765e+00, -6.3823e-01,  1.7839e+00,
          3.4299e-01],
        [-9.6705e-03, -3.0180e+00,  1.3303e+00,  1.6581e+00,  9.0906e-01,
          3.1121e-01,  6.8777e-01,  1.8069e+00,  1.3980e+00,  2.3220e+00,
          7.2656e-01, -1.7658e-02,  3.4216e+00,  2.0307e+00,  9.5702e-01,
          1.1924e+00],

In [89]:
print(Z2.shape)
print(Z2)


torch.Size([10, 6])
tensor([[  4.4474,   9.0393,  11.7886,   7.4195,  -1.7824,   0.9354],
        [-10.7687,  -1.0620,   9.1375,  -1.3888, -16.5643,   3.4996],
        [  1.9857,   4.3433,  -4.3767,  -1.2304, -13.3157,  -5.0138],
        [ -6.7677,  -0.8192,  16.0649,  -0.5587,  -8.9556,  -2.0552],
        [  9.0409,   9.2517,   8.6589,   7.7473,  -1.2442,  -3.8018],
        [  6.7998,  11.4928, -18.2534,  -8.8833,  -7.0860,  -9.1056],
        [  1.6985,   4.5380,   4.7159,   1.5705,  -8.6264,  -3.6949],
        [  1.6218,   3.2332,   4.9810,   1.4529,  -9.6892,  -4.9428],
        [ -3.5950,   8.4994, -10.4533, -11.2190,  -8.9027,  -2.3180],
        [  7.3140,  20.0014,  -4.5374,  -7.4184,  11.3346,  -3.5200]])


In [90]:
print(Z3.shape)
print(Z3)

torch.Size([10, 1])
tensor([[-18.8751],
        [  1.8711],
        [ 17.7120],
        [-27.4697],
        [-19.5918],
        [ 41.9245],
        [ -4.5666],
        [ -6.5606],
        [ 36.3505],
        [  8.5659]])


# Problem 5
- 위에서 구한 `A3[i,j]`의 인덱싱을 통해서 $h_\theta(x^{(3)})$ 을 출력하세요
- 역시 `python`인덱싱은 `0` 부터 시작한다는 것을 주의하세요

In [97]:
#10by4의 x중, 3행.
print("X3 : ", X_batch[2])
#10by4 X 4by16 => 10 by 16 의 3행
print("\nwhile A1 : \n",A1[2])
#10by16 X 16by6 => 10 by 6 의 3행
print("\nwhile A2 : \n",A2[2])
#10by6 X 6by1 => 10 by 1 의 3행
print("\nwhile A3 : \n",A3[2])

print('\n\nh_theta(x3) = ', A3[2])

X3 :  tensor([ 0.3223, -1.2633,  0.3500,  0.3081])

while A1 : 
 tensor([3.4141, 1.7901, 0.6857, 0.2043, 0.2453, 0.0000, 1.4705, 0.8904, 0.0000,
        0.1385, 0.1828, 2.9946, 2.0765, 0.0000, 1.7839, 0.3430])

while A2 : 
 tensor([1.9857, 4.3433, 0.0000, 0.0000, 0.0000, 0.0000])

while A3 : 
 tensor([17.7120])


h_theta(x3) =  tensor([17.7120])


# Problem 6
위에서 공부한 것을 함수로 만들어 보도록 하겠습니다.
아래 한 layer의 선형 변환을 연산하는 class를 만들어 보도록 하죠.

- Class는 `my_linear_layer()`
  - `__init__(self, n_input, n_output)` 함수:
    - `self.W` 변수 초기화: Weight 행렬 `self.W`를 램덤 Gaussian 생성 (차원에 맞는...)
    - `self.b` 변수 초기화: bias 벡터 `self.b`를 모두 `1`인 벡터 생성 (차원에 맞는...)
  - `forward(A)` 함수:
    - 입력: `A`는 sample batch $X$ 또는 전 layer에서 들어오는 입력 batch $A^{[\ell-1]}$을 입력하는 자리
    - return 값
      - `Z` 변수는 $A^{[\ell-1]}$의 선형 변환 값, 즉 $Z^{[\ell]}$



In [107]:
# 답 작성
class my_linear_layer():
    def __init__(self, n_input, n_output):
        self.W = torch.randn(n_input,n_output)
        self.b = torch.ones(n_output)
    
    def forward(self,A):
        Z = torch.matmul(A,self.W) + self.b
        return Z


답을 확인하기 위해서 `n_input=num_feature`과 `n_output = 5` 인 `my_linear_layer` instance 생성

In [108]:
mll = my_linear_layer(num_feature, 5)
mll.forward(X_batch)

tensor([[ 4.9520,  0.8716,  0.3160,  0.3928,  0.2981],
        [-3.9731, -2.5227, -0.9821,  4.9472,  0.4033],
        [ 4.1335,  0.8093,  1.3796, -0.3485, -0.0939],
        [-2.0050,  3.0979,  2.4360,  1.5515,  1.6884],
        [ 8.0919,  3.5451,  2.1944, -2.2603,  0.3218],
        [ 5.5827,  0.8726,  1.5687, -2.0686,  0.3563],
        [ 3.6332,  2.0646,  1.7940, -0.3202,  0.4755],
        [ 3.7843,  2.9165,  2.4434, -0.7991,  0.5491],
        [-1.4134, -2.4246, -0.6840,  2.4040,  0.8423],
        [ 3.3648,  0.7644,  0.3131, -0.7846,  1.9466]])

In [109]:
print(mll.W)
print(mll.b)

tensor([[-1.3997, -1.2454, -0.1561,  0.4912, -0.3592],
        [-2.4972,  0.5565,  0.1907,  0.6908,  0.8331],
        [ 0.4281,  1.6396,  1.2571, -0.6234, -0.1717],
        [ 0.9086,  1.1028,  0.7490, -1.3499,  0.4362]])
tensor([1., 1., 1., 1., 1.])


# Building a Linear Layer with `torch.nn`
위에서 수행한 작업을 `pytorch`에서는 `torch.nn.Linear`라는 명령어로 쉽게 구현할 수 있습니다.
아래 예제를 보도록 하죠

In [110]:
import torch.nn as nn
#W = torch.randn(num_feature, 5)
L1 = nn.Linear(num_feature, 5)
Zll = L1(X_batch)
Zll

tensor([[-0.1012,  0.0172, -1.0387,  0.3492,  0.2874],
        [-0.6225, -0.2820, -0.0916,  0.3693,  1.0691],
        [-0.3691, -0.2998, -0.6143,  0.4438,  0.2185],
        [ 0.0691,  0.0023, -0.0104,  0.0430, -1.0094],
        [ 0.0245,  0.0657, -1.2511,  0.2203, -0.5423],
        [ 0.3005, -0.4517, -0.0030,  0.9542,  0.0040],
        [-0.1458, -0.1183, -0.6407,  0.2854, -0.2747],
        [-0.1538, -0.1013, -0.6535,  0.1932, -0.5729],
        [ 0.2093, -0.4967,  0.6286,  1.0444,  0.8294],
        [ 1.3057, -0.1712,  0.5252,  1.2051, -0.2927]],
       grad_fn=<AddmmBackward0>)

In [111]:
L1.weight = nn.Parameter(W.T)
L1.bias.data.fill_(1.0)
Zll2 = L1(X_batch)
Zll2

tensor([[ 2.9194,  0.6712,  0.3797,  3.0094,  0.8064],
        [ 1.4650,  4.0738, -0.1087,  1.9831,  2.7174],
        [ 1.5559,  0.7344,  2.4332,  2.6213,  0.5947],
        [-1.6455,  1.2050,  2.0044, -1.4197,  2.7569],
        [ 1.8688, -1.0644,  2.1738,  2.3409,  0.3636],
        [ 2.0451, -0.6727,  2.3095,  2.3790, -1.5663],
        [ 0.9045,  0.3954,  2.1593,  1.5614,  1.0787],
        [ 0.2249,  0.0947,  2.7885,  1.0601,  1.2692],
        [ 2.2619,  2.1101, -0.1008,  1.9378, -0.2486],
        [ 2.8238, -0.9667, -0.8175,  0.9938, -2.1230]],
       grad_fn=<AddmmBackward0>)

In [112]:
Z

tensor([[ 2.9194,  0.6712,  0.3797,  3.0094,  0.8064],
        [ 1.4650,  4.0738, -0.1087,  1.9831,  2.7174],
        [ 1.5559,  0.7344,  2.4332,  2.6213,  0.5947],
        [-1.6455,  1.2050,  2.0044, -1.4197,  2.7569],
        [ 1.8688, -1.0644,  2.1738,  2.3409,  0.3636],
        [ 2.0451, -0.6727,  2.3095,  2.3790, -1.5663],
        [ 0.9045,  0.3954,  2.1593,  1.5614,  1.0787],
        [ 0.2249,  0.0947,  2.7885,  1.0601,  1.2692],
        [ 2.2619,  2.1101, -0.1008,  1.9378, -0.2486],
        [ 2.8238, -0.9667, -0.8175,  0.9938, -2.1230]])